# 政府公開資訊網站氣象觀測資料自動化爬取

In [2]:
pip install apscheduler

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
import csv
from apscheduler.schedulers.blocking import BlockingScheduler

In [18]:
headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
    }
API_URL = 'https://opendata.cwb.gov.tw/api/v1/rest/datastore/O-A0001-001?Authorization=rdec-key-123-45678-011121314'
resp = requests.get(API_URL, headers=headers)
data = resp.json()
data

{'success': 'true',
 'result': {'resource_id': 'O-A0001-001',
  'fields': [{'id': 'lat', 'type': 'Double'},
   {'id': 'lon', 'type': 'Double'},
   {'id': 'locationName', 'type': 'String'},
   {'id': 'stationId', 'type': 'String'},
   {'id': 'obsTime', 'type': 'Timestamp'},
   {'id': 'elementName', 'type': 'String'},
   {'id': 'elementValue', 'type': 'String'},
   {'id': 'parameterName', 'type': 'String'},
   {'id': 'parameterValue', 'type': 'String'}]},
 'records': {'location': [{'lat': '24.778333',
    'lon': '121.494583',
    'locationName': '福山',
    'stationId': 'C0A560',
    'time': {'obsTime': '2023-05-04 23:00:00'},
    'weatherElement': [{'elementName': 'ELEV', 'elementValue': '405.0'},
     {'elementName': 'WDIR', 'elementValue': '0'},
     {'elementName': 'WDSD', 'elementValue': '0.0'},
     {'elementName': 'TEMP', 'elementValue': '23.3'},
     {'elementName': 'HUMD', 'elementValue': '0.96'},
     {'elementName': 'PRES', 'elementValue': '962.5'},
     {'elementName': 'H_24R',

In [19]:
location_records = data['records']['location']
location_records

[{'lat': '24.778333',
  'lon': '121.494583',
  'locationName': '福山',
  'stationId': 'C0A560',
  'time': {'obsTime': '2023-05-04 23:00:00'},
  'weatherElement': [{'elementName': 'ELEV', 'elementValue': '405.0'},
   {'elementName': 'WDIR', 'elementValue': '0'},
   {'elementName': 'WDSD', 'elementValue': '0.0'},
   {'elementName': 'TEMP', 'elementValue': '23.3'},
   {'elementName': 'HUMD', 'elementValue': '0.96'},
   {'elementName': 'PRES', 'elementValue': '962.5'},
   {'elementName': 'H_24R', 'elementValue': '0.0'},
   {'elementName': 'H_FX', 'elementValue': '-99'},
   {'elementName': 'H_XD', 'elementValue': '-99'},
   {'elementName': 'H_FXT', 'elementValue': '-99'},
   {'elementName': 'D_TX', 'elementValue': '31.50'},
   {'elementName': 'D_TXT', 'elementValue': '2023-05-04T11:30:00+08:00'},
   {'elementName': 'D_TN', 'elementValue': '20.20'},
   {'elementName': 'D_TNT', 'elementValue': '2023-05-04T06:10:00+08:00'},
   {'elementName': 'Weather', 'elementValue': '陰'}],
  'parameter': [{'p

In [20]:
row_list = []

for location_record in location_records:
        # 從 JSON檔轉成 dict/list 中取值
    lat = location_record['lat']
    lon = location_record['lon']
    location_name = location_record['locationName']
    station_id = location_record['stationId']
    time = location_record['time']['obsTime']
    ELE = location_record['weatherElement'][0]['elementValue']
    WDIR = location_record['weatherElement'][1]['elementValue']
    WDSD = location_record['weatherElement'][2]['elementValue']
    TEMP = location_record['weatherElement'][3]['elementValue']
    HUMD = location_record['weatherElement'][4]['elementValue']
    PRES = location_record['weatherElement'][5]['elementValue']
        
        # 將資料整理成dict
    data2 = {}
    data2['lat'] = lat
    data2['lon'] = lon
    data2['locationName'] = location_name
    data2['stationId'] = station_id
    data2['obstime'] = time
    data2['ELE'] = ELE
    data2['WDIR'] = WDIR
    data2['WDSD'] = WDSD
    data2['TEMP'] = TEMP
    data2['HUMD'] = HUMD
    data2['PRES'] = PRES

    row_list.append(data2) #存入 row_list 方便之後寫入 csv 檔案使用
    
row_list   

[{'lat': '24.778333',
  'lon': '121.494583',
  'locationName': '福山',
  'stationId': 'C0A560',
  'obstime': '2023-05-04 23:00:00',
  'ELE': '405.0',
  'WDIR': '0',
  'WDSD': '0.0',
  'TEMP': '23.3',
  'HUMD': '0.96',
  'PRES': '962.5'},
 {'lat': '22.994972',
  'lon': '120.144111',
  'locationName': '安平',
  'stationId': 'C0X190',
  'obstime': '2023-05-04 23:00:00',
  'ELE': '10.0',
  'WDIR': '74',
  'WDSD': '1.2',
  'TEMP': '26.7',
  'HUMD': '0.83',
  'PRES': '1006.3'},
 {'lat': '22.956258',
  'lon': '121.221844',
  'locationName': '七塊厝',
  'stationId': 'C0S950',
  'obstime': '2023-05-04 23:00:00',
  'ELE': '415.0',
  'WDIR': '238',
  'WDSD': '1.8',
  'TEMP': '21.9',
  'HUMD': '0.96',
  'PRES': '963.8'},
 {'lat': '25.019167',
  'lon': '121.934167',
  'locationName': '福隆',
  'stationId': 'C0A880',
  'obstime': '2023-05-04 23:00:00',
  'ELE': '6.0',
  'WDIR': '172',
  'WDSD': '2.5',
  'TEMP': '26.0',
  'HUMD': '0.87',
  'PRES': '1005.7'},
 {'lat': '24.133550',
  'lon': '121.287556',
  'loc

In [21]:
def get_api_data(API_URL): #爬資料
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
    }
    # 發出網路請求
    resp = requests.get(API_URL, headers=headers)
    data = resp.json() #將回傳值從 JSON 格式轉成 Python dict 字典格式方便存取
    return data

def get_parse_data(data): #把資料從JSON檔取值存到空list
    location_records = data['records']['location']
    row_list = []

    for location_record in location_records:
        # 從 JSON檔轉成 dict/list 中取值
        lat = location_record['lat']
        lon = location_record['lon']
        location_name = location_record['locationName']
        station_id = location_record['stationId']
        time = location_record['time']['obsTime']
        ELE = location_record['weatherElement'][0]['elementValue']
        WDIR = location_record['weatherElement'][1]['elementValue']
        WDSD = location_record['weatherElement'][2]['elementValue']
        TEMP = location_record['weatherElement'][3]['elementValue']
        HUMD = location_record['weatherElement'][4]['elementValue']
        PRES = location_record['weatherElement'][5]['elementValue']
        
        # 將資料整理成dict
        data2 = {}
        data2['lat'] = lat
        data2['lon'] = lon
        data2['locationName'] = location_name
        data2['stationId'] = station_id
        data2['obstime'] = time
        data2['ELE'] = ELE
        data2['WDIR'] = WDIR
        data2['WDSD'] = WDSD
        data2['TEMP'] = TEMP
        data2['HUMD'] = HUMD
        data2['PRES'] = PRES

        row_list.append(data2) #存入 row_list 方便之後寫入 csv 檔案使用

    return row_list

def save_data_to_csv(row_list):
    #CSV 檔案第一列標題記得要和 dict 的 key 相同，不然會出錯
    headers = ['lat', 'lon', 'locationName', 'stationId', 'obstime', 'ELE', 'WDIR', 'WDSD', 'TEMP', 'HUMD', 'PRES']
    with open('C:/Users/jason/OneDrive/桌面/happycoder/weather.csv', 'w') as output_file:
        dict_writer = csv.DictWriter(output_file, headers)
        dict_writer.writeheader()
        dict_writer.writerows(row_list)

# 創建一個 Scheduler 物件實例
sched = BlockingScheduler({'apscheduler.timezone': 'Asia/Taipei'})

# decorator 設定 Scheduler 的類型和參數，例如 interval 間隔多久執行
@sched.scheduled_job('interval', seconds=10)
def timed_job():
    print('每 10 秒執行一次程式工作區塊')
    API_URL = 'https://opendata.cwb.gov.tw/api/v1/rest/datastore/O-A0001-001?Authorization=rdec-key-123-45678-011121314'
    data = get_api_data(API_URL)
    row_list = get_parse_data(data)
    save_data_to_csv(row_list)

# 開始執行
sched.start()

每 10 秒執行一次程式工作區塊
每 10 秒執行一次程式工作區塊


KeyboardInterrupt: 